In [1]:
import keras
import tensorflow as tf

print(tf.__version__)
print(keras.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-04-01 01:41:50.076549: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-01 01:41:50.105739: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-01 01:41:50.105771: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-01 01:41:50.106799: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-01 01:41:50.112085: I tensorflow/core/platform/cpu_feature_guar

2.15.1
2.15.0
Num GPUs Available:  1


2024-04-01 01:41:51.243411: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-01 01:41:51.273484: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-01 01:41:51.273635: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
import numpy as np
from skimage import io
from keras import backend as K
import segmentation_models_3D as sm
from matplotlib import pyplot as plt

from tensorflow import data as tf_data
from tensorflow import image as tf_image
from tensorflow import io as tf_io
from skimage import io
import math


Segmentation Models: using `tf.keras` framework.


In [13]:
class Gen(tf.keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

        return np.array([
            np.stack((io.imread(file_name_x),)*3, axis=-1)
               for file_name_x in batch_x]), np.array([
            io.imread(file_name_y).astype(np.float32)
               for file_name_y in batch_y])

In [14]:

# def get_dataset(
#     batch_size,
#     input_img_paths,
#     target_img_paths,
#     max_dataset_len=None,
# ):
#     """Returns a TF Dataset."""

#     def load_img_masks(input_img_path, target_img_path):
#         input_img = io.imread(input_img_path)
#         input_img = np.stack((input_img,)*3, axis=-1)
#         input_img = tf_image.convert_image_dtype(input_img, "float32")

#         target_img = io.imread(target_img_path)
#         target_img = tf_image.convert_image_dtype(target_img, "float32")

#         # Ground truth labels are 1, 2, 3. Subtract one to make them 0, 1, 2:
#         target_img -= 1
#         return input_img, target_img

#     # For faster debugging, limit the size of data
#     if max_dataset_len:
#         input_img_paths = input_img_paths[:max_dataset_len]
#         target_img_paths = target_img_paths[:max_dataset_len]
#     dataset = tf_data.Dataset.from_tensor_slices((input_img_paths, target_img_paths))
#     dataset = dataset.map(load_img_masks, num_parallel_calls=tf_data.AUTOTUNE)
#     return dataset.batch(batch_size)

In [15]:
# import random
# import os
# from glob import glob

# input_img_paths = glob(os.path.join('data', 'patches', 'images', '*.tif'))
# input_img_paths.sort()
# target_img_paths = glob(os.path.join('data', 'patches', 'masks', '*.tif'))
# target_img_paths.sort()

# batch_size = 16
# patch_size = (32, 128, 128)

# # Split our img paths into a training and a validation set
# val_samples = 1000
# random.Random(1337).shuffle(input_img_paths)
# random.Random(1337).shuffle(target_img_paths)
# train_input_img_paths = input_img_paths[:-val_samples]
# train_target_img_paths = target_img_paths[:-val_samples]
# val_input_img_paths = input_img_paths[-val_samples:]
# val_target_img_paths = target_img_paths[-val_samples:]

# # Instantiate dataset for each split
# # Limit input files in `max_dataset_len` for faster epoch training time.
# # Remove the `max_dataset_len` arg when running with full dataset.
# train_dataset = get_dataset(
#     batch_size,
#     train_input_img_paths,
#     train_target_img_paths,
#     max_dataset_len=1000,
# )
# valid_dataset = get_dataset(
#     batch_size, val_input_img_paths, val_target_img_paths
# )

In [16]:
import random
import os
from glob import glob

input_img_paths = glob(os.path.join('data', 'patches', 'images', '*.tif'))
input_img_paths.sort()
target_img_paths = glob(os.path.join('data', 'patches', 'masks', '*.tif'))
target_img_paths.sort()

batch_size = 16
patch_size = (32, 128, 128)

data_gen = Gen(input_img_paths, target_img_paths, batch_size)

In [17]:

# Loss Function and coefficients to be used during training:
def dice_coefficient(y_true, y_pred):
    smoothing_factor = 1
    flat_y_true = K.flatten(y_true)
    flat_y_pred = K.flatten(y_pred)
    return (2. * K.sum(flat_y_true * flat_y_pred) + smoothing_factor) / (K.sum(flat_y_true) + K.sum(flat_y_pred) + smoothing_factor)

def dice_coefficient_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)


Backbones: ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'seresnet18', 'seresnet34', 'seresnet50', 'seresnet101', 'seresnet152', 'seresnext50', 'seresnext101', 'senet154', 'resnext50', 'resnext101', 'vgg16', 'vgg19', 'densenet121', 'densenet169', 'densenet201', 'inceptionresnetv2', 'inceptionv3', 'mobilenet', 'mobilenetv2', 'efficientnetb0', 'efficientnetb1', 'efficientnetb2', 'efficientnetb3', 'efficientnetb4', 'efficientnetb5', 'efficientnetb6', 'efficientnetb7']

In [18]:
#Define parameters for our model.

encoder_weights = 'imagenet'
BACKBONE = 'resnet50'  #Try vgg16, efficientnetb7, inceptionv3, resnet50
activation = 'softmax'
n_classes = 2
channels = 3
epochs = 1000

LR = 1e-5
optim = keras.optimizers.AdamW(LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
# set class weights for dice_loss (car: 1.; pedestrian: 2.; background: 0.5;)
dice_loss = sm.losses.DiceLoss() 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

In [19]:
preprocess_input = sm.get_preprocessing(BACKBONE)

In [20]:
#Define the model. Here we use Unet but we can also use other model architectures from the library.
model = sm.Unet(BACKBONE, classes=n_classes, 
                input_shape=(patch_size[0], patch_size[1], patch_size[2], channels), 
                encoder_weights=encoder_weights,
                activation=activation)

model.compile(optimizer = optim, loss=total_loss, metrics=metrics)
print(model.summary())

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 data (InputLayer)           [(None, 32, 128, 128, 3)]    0         []                            
                                                                                                  
 bn_data (BatchNormalizatio  (None, 32, 128, 128, 3)      9         ['data[0][0]']                
 n)                                                                                               
                                                                                                  
 zero_padding3d_18 (ZeroPad  (None, 38, 134, 134, 3)      0         ['bn_data[0][0]']             
 ding3D)                                                                                          
                                                                                            

In [21]:
#Fit the model

callbacks = [
    keras.callbacks.ModelCheckpoint("unet3d_segmentation.keras", save_best_only=True)
]

history = model.fit(data_gen, 
          batch_size=batch_size, 
          epochs=epochs,
          callbacks=callbacks,
          verbose=1,
)

Epoch 1/1000


2024-04-01 01:43:48.901419: I tensorflow/core/grappler/optimizers/generic_layout_optimizer.cc:403] Cancel Transpose nodes around Pad: transpose_before=model_3/bn_data/FusedBatchNormV3-0-1-TransposeNCDHWToNDHWC-LayoutOptimizer pad=model_3/zero_padding3d_18/Pad transpose_after=model_3/conv0/Conv3D-0-TransposeNDHWCToNCDHW-LayoutOptimizer,gradient_tape/model_3/conv0/Conv3D/Conv3DBackpropFilterV2-0-TransposeNDHWCToNCDHW-LayoutOptimizer,gradient_tape/model_3/conv0/Conv3D/ShapeN-0-TransposeNDHWCToNCDHW-LayoutOptimizer
2024-04-01 01:43:50.376355: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-04-01 01:43:50.428618: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-01 01:43:50.590373: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-01 01:43:54.183571: W external/local_tsl/tsl/framework/bfc_a

ResourceExhaustedError: Graph execution error:

Detected at node model_3/stage2_unit1_bn2/FusedBatchNormV3 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/asebaq/dev/cell_segmentation/venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/asebaq/dev/cell_segmentation/venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/asebaq/dev/cell_segmentation/venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/asebaq/dev/cell_segmentation/venv/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", lin

In [ ]:
#Save model for future use
model.save(f'3D_model_{BACKBONE}_ep{epochs}.h5')

In [ ]:
###
#plot the training and validation IoU and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['iou_score']
val_acc = history.history['val_iou_score']

plt.plot(epochs, acc, 'y', label='Training IOU')
plt.plot(epochs, val_acc, 'r', label='Validation IOU')
plt.title('Training and validation IOU')
plt.xlabel('Epochs')
plt.ylabel('IOU')
plt.legend()
plt.show()

In [ ]:
#Load the pretrained model for testing and predictions. 
from keras.models import load_model
my_model = load_model('3D_model_vgg16_200epochs.h5', compile=False)
#If you load a different model do not forget to preprocess accordingly. 

In [ ]:
#Predict on the test data
y_pred=my_model.predict(X_test)
y_pred_argmax=np.argmax(y_pred, axis=4)
y_test_argmax = np.argmax(y_test, axis=4)


In [ ]:
print(y_pred_argmax.shape)
print(y_test_argmax.shape)
print(np.unique(y_pred_argmax))

In [ ]:
#Using built in keras function for IoU
#Only works on TF > 2.0
from keras.metrics import MeanIoU
from keras.metrics import MeanIoU
n_classes = 2
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(y_test_argmax, y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

In [ ]:
#Test some random images
import random
test_img_number = random.randint(0, len(X_test))
test_img = X_test[test_img_number]
ground_truth=y_test[test_img_number]

test_img_input=np.expand_dims(test_img, 0)
test_img_input1 = preprocess_input(test_img_input)

test_pred1 = my_model.predict(test_img_input1)
test_prediction1 = np.argmax(test_pred1, axis=4)[0,:,:,:]
print(test_prediction1.shape)

In [ ]:
ground_truth_argmax = np.argmax(ground_truth, axis=3)
print(test_img.shape)

In [ ]:
#Plot individual slices from test predictions for verification
slice = 14
plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img[slice,:,:,0], cmap='gray')
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth_argmax[slice,:,:])
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(test_prediction1[slice,:,:])
plt.show()

Now segment the full volume using the trained model.

In [ ]:
#Break the large image (volume) into patches of same size as the training images (patches)
large_image = io.imread('data/Fluo-N3DH-SIM+/02/t000.tif')
patches = patchify(large_image, (32, 32, 32), step=32)  #Step=256 for 256 patches means no overlap
print(large_image.shape)
print(patches.shape)

In [ ]:
# Predict each 3D patch   
predicted_patches = []
for i in range(patches.shape[0]):
  for j in range(patches.shape[1]):
    for k in range(patches.shape[2]):
      #print(i,j,k)
      single_patch = patches[i,j,k, :,:,:]
      single_patch_3ch = np.stack((single_patch,)*3, axis=-1)
      single_patch_3ch_input = preprocess_input(np.expand_dims(single_patch_3ch, axis=0))
      single_patch_prediction = my_model.predict(single_patch_3ch_input)
      single_patch_prediction_argmax = np.argmax(single_patch_prediction, axis=4)[0,:,:,:]
      predicted_patches.append(single_patch_prediction_argmax)
      


In [ ]:
#Convert list to numpy array
predicted_patches = np.array(predicted_patches)
print(predicted_patches.shape)

In [ ]:
#Reshape to the shape we had after patchifying
predicted_patches_reshaped = np.reshape(predicted_patches, 
                                        (patches.shape[0], patches.shape[1], patches.shape[2],
                                         patches.shape[3], patches.shape[4], patches.shape[5]) )
print(predicted_patches_reshaped.shape)

In [ ]:
#Repach individual patches into the orginal volume shape
reconstructed_image = unpatchify(predicted_patches_reshaped, large_image.shape)
print(reconstructed_image.shape)

In [ ]:
print(reconstructed_image.dtype)

In [ ]:
#Convert to uint8 so we can open image in most image viewing software packages
reconstructed_image=reconstructed_image.astype(np.uint8)
print(reconstructed_image.dtype)

In [ ]:
#Now save it as segmented volume.
from tifffile import imsave
imsave('/content/drive/MyDrive/Colab Notebooks/data/sandstone_3d/all_images/segmented.tif', reconstructed_image)
    

In [ ]:
#If you would like to save the volume as multichannel dataset....

In [ ]:
print(np.unique(reconstructed_image))

In [ ]:
#Seperate each channel/segment to be combined as multiple channels.
num_segments=4
segm0 = (reconstructed_image == 0)
segm1 = (reconstructed_image == 1)
segm2 = (reconstructed_image == 2)
segm3 = (reconstructed_image == 3)

final = np.empty((reconstructed_image.shape[0], reconstructed_image.shape[1], reconstructed_image.shape[2], num_segments))
final[:,:,:,0] = segm0
final[:,:,:,1] = segm1
final[:,:,:,2] = segm2
final[:,:,:,3] = segm3

In [ ]:
#Use APEER OMETIFF library to read and write multidimensional images
!pip install apeer-ometiff-library

In [ ]:
from apeer_ometiff_library import io

In [ ]:
# Expand image array to 5D of order (T, Z, C, X, Y)
# This is the convention for OMETIFF format as written by APEER library
final = np.expand_dims(final, axis=0)
final=np.swapaxes(final, 2, 4)

final = final.astype(np.int8)



In [ ]:
print("Shape of the segmented volume is: T, Z, C, X, Y ", final.shape)
print(final.dtype)


In [ ]:
# Write dataset as multi-dimensional OMETIFF *image*
io.write_ometiff("segmented_multi_channel.ome.tiff", final)